# Python整理資料實務1 — 透過快速新增多個欄位比較兩資料集的差異

此篇為Python整理資料實務第一篇 ([文章](https://peisyuanli.medium.com/python%E6%95%B4%E7%90%86%E8%B3%87%E6%96%99%E5%AF%A6%E5%8B%99-%E9%80%8F%E9%81%8E%E5%BF%AB%E9%80%9F%E6%96%B0%E5%A2%9E%E5%A4%9A%E5%80%8B%E6%AC%84%E4%BD%8D%E6%AF%94%E8%BC%83%E5%85%A9%E8%B3%87%E6%96%99%E9%9B%86%E7%9A%84%E5%B7%AE%E7%95%B0-25dc5b1f05d5))使用的程式碼。

網路上很多關於Data Frame的相關教學並統整常用的method，實務上資料處理可能多變萬化，本篇文章藉由快速新增多個欄位比較兩資料集的多個欄位是否有相異，並示範資料整理常用的技巧，包含合併兩資料及和一次新增多個資料欄位。實務上遇到的資料通常比較複雜且欄位屬性更多，以簡化的範例來說明。

## 情境和資料
實務上常會需要比對兩資料集數值是否相同，遇到的資料通常比較複雜且欄位數量更多，本篇文章以簡化的範例來說明。
假設資料從來源1 (資料庫1) 在固定時間更新至來源2 (資料庫2)，但兩資料中的部分數值不一致，已知欄位數量相同且資料筆數相同，如何找出數值不相同的資料列，來釐清問題。

範例資料為某產品的業務預估銷售資料，欄位分別有VERSION (預估版本)、YEAR (年)、MONTH (月)、CUSTOMER (客戶)、MODEL (型號)、SITE (工廠代號)、BU (事業處)、QTY (數量)、REVENUE (營收)和MARGIN (毛利)。將針對QTY (數量)、REVENUE (營收)和MARGIN (毛利)進行數值比對。其中範例資料中的數值為非實際數值。

### Step 1: 讀資料

In [1]:
import pandas as pd
file = 'data_example.xlsx'
df_source1 = pd.read_excel( file, sheet_name = 'source1')
df_source2 = pd.read_excel( file, sheet_name = 'source2')

In [2]:
df_source1

,VERSION,YEAR,MONTH,CUSTOMER,MODEL,SITE,BU,QTY,REVENUE,MARGIN
0,V_20210108,2021,1,A,M01,SITE3,BU1,486,3307,300
1,V_20210108,2021,2,A,M01,SITE3,BU1,511,3500,356
2,V_20210108,2021,1,A,M01,SITE1,BU1,745,5166,310
3,V_20210108,2021,2,A,M01,SITE1,BU1,733,5100,288
4,V_20210108,2021,1,A,M02,SITE2,BU1,255,3841,612
5,V_20210108,2021,2,A,M02,SITE2,BU1,300,3900,700
6,V_20210108,2021,1,B,M03,SITE3,BU2,632,8652,1536
7,V_20210108,2021,2,B,M03,SITE3,BU2,700,8745,1666
8,V_20210108,2021,1,C,M04,SITE2,BU3,100,520,53
9,V_20210108,2021,2,C,M04,SITE2,BU3,120,532,77


In [3]:
df_source2

,VERSION,YEAR,MONTH,CUSTOMER,MODEL,SITE,BU,QTY,REVENUE,MARGIN
0,V_20210108,2021,1,A,M01,SITE3,BU1,486,3307,300
1,V_20210108,2021,2,A,M01,SITE3,BU1,511,3500,366
2,V_20210108,2021,1,A,M01,SITE1,BU1,745,5166,310
3,V_20210108,2021,2,A,M01,SITE1,BU1,733,5100,312
4,V_20210108,2021,1,A,M02,SITE2,BU1,255,3841,612
5,V_20210108,2021,2,A,M02,SITE2,BU1,300,3900,700
6,V_20210108,2021,1,B,M03,SITE3,BU2,632,8652,1536
7,V_20210108,2021,2,B,M03,SITE3,BU2,700,8745,1789
8,V_20210108,2021,1,C,M04,SITE2,BU3,100,520,53
9,V_20210108,2021,2,C,M04,SITE2,BU3,120,532,120


### Step 2: 合併兩資料集

In [4]:
df_comb = df_source1.merge( df_source2, on = ['VERSION', 'YEAR', 'MONTH', 'CUSTOMER', 'MODEL', 'SITE', 'BU'], how = 'inner',
                            suffixes=('_1', '_2') ); df_comb
# 合併的key為VERSION、YEAR、MONTH、CUSTOMER、MODEL、SITE和BU等7個欄位
# 用字尾 (suffix)區分兩個資料來源，若為資料來源1，則字尾為"_1"

,VERSION,YEAR,MONTH,CUSTOMER,MODEL,SITE,BU,QTY_1,REVENUE_1,MARGIN_1,QTY_2,REVENUE_2,MARGIN_2
0,V_20210108,2021,1,A,M01,SITE3,BU1,486,3307,300,486,3307,300
1,V_20210108,2021,2,A,M01,SITE3,BU1,511,3500,356,511,3500,366
2,V_20210108,2021,1,A,M01,SITE1,BU1,745,5166,310,745,5166,310
3,V_20210108,2021,2,A,M01,SITE1,BU1,733,5100,288,733,5100,312
4,V_20210108,2021,1,A,M02,SITE2,BU1,255,3841,612,255,3841,612
5,V_20210108,2021,2,A,M02,SITE2,BU1,300,3900,700,300,3900,700
6,V_20210108,2021,1,B,M03,SITE3,BU2,632,8652,1536,632,8652,1536
7,V_20210108,2021,2,B,M03,SITE3,BU2,700,8745,1666,700,8745,1789
8,V_20210108,2021,1,C,M04,SITE2,BU3,100,520,53,100,520,53
9,V_20210108,2021,2,C,M04,SITE2,BU3,120,532,77,120,532,120


**一般來說，增加資料行的方式為下，在此例中比較的項目為QTY、REVENUE 和 MARGIN 共三項。**

**當要比較的項目數量較多時，只能一行一行產生嗎?**

In [5]:
df_comb['QTY_DEL'] = df_comb['QTY_1'] - df_comb[ 'QTY_2' ]  
df_comb['REVENUE_DEL'] = df_comb['REVENUE_1'] - df_comb[ 'REVENUE_2' ]  
df_comb['MARGIN_DEL'] = df_comb['MARGIN_1'] - df_comb[ 'MARGIN_2' ]

### Step 3: 定義要計算的項目

In [6]:
list_to_cal = ['QTY', 'REVENUE', 'MARGIN' ]

### Step 4: 透過格式化和迴圈新增多個資料欄位

In [7]:
for m in list_to_cal:
    # 針對每個項目，找出兩個要相減的欄位，以來源1的數值對來源2相減
    target = df_comb.columns[ df_comb.columns.str.startswith(m) ]
    df_comb['%s_DEL'%m] = df_comb[ target[0] ] - df_comb[ target[1] ]
df_comb

,VERSION,YEAR,MONTH,CUSTOMER,MODEL,SITE,BU,QTY_1,REVENUE_1,MARGIN_1,QTY_2,REVENUE_2,MARGIN_2,QTY_DEL,REVENUE_DEL,MARGIN_DEL
0,V_20210108,2021,1,A,M01,SITE3,BU1,486,3307,300,486,3307,300,0,0,0
1,V_20210108,2021,2,A,M01,SITE3,BU1,511,3500,356,511,3500,366,0,0,-10
2,V_20210108,2021,1,A,M01,SITE1,BU1,745,5166,310,745,5166,310,0,0,0
3,V_20210108,2021,2,A,M01,SITE1,BU1,733,5100,288,733,5100,312,0,0,-24
4,V_20210108,2021,1,A,M02,SITE2,BU1,255,3841,612,255,3841,612,0,0,0
5,V_20210108,2021,2,A,M02,SITE2,BU1,300,3900,700,300,3900,700,0,0,0
6,V_20210108,2021,1,B,M03,SITE3,BU2,632,8652,1536,632,8652,1536,0,0,0
7,V_20210108,2021,2,B,M03,SITE3,BU2,700,8745,1666,700,8745,1789,0,0,-123
8,V_20210108,2021,1,C,M04,SITE2,BU3,100,520,53,100,520,53,0,0,0
9,V_20210108,2021,2,C,M04,SITE2,BU3,120,532,77,120,532,120,0,0,-43
